# DATA VISUALIZATION

In [1]:
pip install jupyter-dash


     ---------------------------------------- 7.8/7.8 MB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load the data
data = pd.read_csv(r"C:\Users\grove\Desktop\updated_investor_data.csv")

# Initialize the Dash app
app = Dash(__name__)

# Layout for the dashboard
app.layout = html.Div([
    html.H1("Investment Dashboard", style={"textAlign": "center"}),

    # Dropdown for segmentation insights
    html.Div([
        html.Label("Select Segmentation Feature:"),
        dcc.Dropdown(
            id='segmentation-dropdown',
            options=[
                {"label": "Age Group", "value": "Age_Group"},
                {"label": "Income Level", "value": "Income_Level"},
                {"label": "Risk Tolerance", "value": "Risk_Tolerance"},
                {"label": "Preferred Investment", "value": "Preferred_Investment"}
            ],
            value="Age_Group"
        )
    ]),
    dcc.Graph(id='segmentation-graph'),

    # Investment recommendations
    html.Div([
        html.Label("Preferred Investment Type:"),
        dcc.Dropdown(
            id='investment-type-dropdown',
            options=[
                {"label": inv, "value": inv} for inv in data['Preferred_Investment'].unique()
            ],
            value=data['Preferred_Investment'].unique()[0]
        )
    ]),
    dcc.Graph(id='investment-recommendation-graph'),

    # Portfolio performance monitoring
    html.Div([
        html.H3("Portfolio Performance and Liquidity", style={"textAlign": "center"}),
        dcc.Graph(id='portfolio-performance-graph',
                  figure=px.scatter(
                      data,
                      x="Portfolio_to_Net_Worth_Ratio",
                      y="Risk_Adjusted_Returns_(Percent)",
                      color="Liquidity_Preference",
                      size="Portfolio_Value",
                      hover_data=['Name'],
                      title="Portfolio Performance by Liquidity Preference"
                  ))
    ])
])

# Callback for segmentation insights
@app.callback(
    Output('segmentation-graph', 'figure'),
    Input('segmentation-dropdown', 'value')
)
def update_segmentation_graph(feature):
    fig = px.histogram(
        data, 
        x=feature, 
        color="Risk_Tolerance",
        barmode='group',
        title=f"Segmentation Insights by {feature}"
    )
    return fig


@app.callback(
    Output('investment-recommendation-graph', 'figure'),
    Input('investment-type-dropdown', 'value')
)
def update_investment_graph(investment_type):
    filtered_data = data[data['Preferred_Investment'] == investment_type]
    fig = px.bar(
        filtered_data,
        x="Name",
        y="Risk_Adjusted_Returns_(Percent)",
        color="Risk_Tolerance",
        title=f"Investment Recommendations for {investment_type}"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


# The END